# Sentiment Analysis of News

## Dictionary-based Approach

In [5]:
from nltk.tokenize import treebank
from nltk.corpus import opinion_lexicon

import pandas as pd

In [6]:
nyt = pd.read_csv('csv_files/nyt.csv')
nyt.head()

,snippet,abstract,pub_date,_id,section_name,uri,type_of_material,source,print_section,lead_paragraph,word_count,news_desk,web_url,document_type,print_page,headline,author
0,Mexican lawmakers reach compromise on country'...,Mexican lawmakers reach compromise on country'...,1998-12-31T05:00:00+0000,nyt://article/6238be98-a87a-5fbd-8a5d-20b15341...,World,nyt://article/6238be98-a87a-5fbd-8a5d-20b15341...,News,The New York Times,A,After weeks of tense impasse and with less tha...,518,Foreign Desk,https://www.nytimes.com/1998/12/31/world/just-...,article,9,"Just Hours Before Deadline, Mexicans Draft an ...",By Ginger Thompson
1,Three new studies report new technique to pinp...,Three new studies report new technique to pinp...,1998-12-29T05:00:00+0000,nyt://article/bd31e0fe-4053-55f8-9e24-86b10281...,Science,nyt://article/bd31e0fe-4053-55f8-9e24-86b10281...,News,The New York Times,F,"Every winter, after flying south from the East...",1170,Science Desk,https://www.nytimes.com/1998/12/29/science/on-...,article,5,"On the Trail of the Monarch, With the Aid of C...",By Carol Kaesuk Yoon
2,Interview with Lieut Col Hildegardo Bacillo Go...,Interview with Lieut Col Hildegardo Bacillo Go...,1998-12-29T05:00:00+0000,nyt://article/fc8cfb88-aa41-5ec5-8328-f2e6d870...,World,nyt://article/fc8cfb88-aa41-5ec5-8328-f2e6d870...,News,The New York Times,A,Sheltered from view by bushes on the side of t...,1352,Foreign Desk,https://www.nytimes.com/1998/12/29/world/mexic...,article,3,Mexican Army Protester Goes Loudly Into Hiding,By Ginger Thompson
3,"Some 2,000 people gather at graves of 45 peopl...","Some 2,000 people gather at graves of 45 peopl...",1998-12-23T05:00:00+0000,nyt://article/1eaf78c0-88df-5390-bd95-664fcf07...,World,nyt://article/1eaf78c0-88df-5390-bd95-664fcf07...,News,The New York Times,A,"For six days, only the dogs and chickens were ...",951,Foreign Desk,https://www.nytimes.com/1998/12/23/world/actea...,article,4,Acteal Journal; Where Killings Defiled a Churc...,By Ginger Thompson
4,Ambitious United States effort to help train a...,Ambitious United States effort to help train a...,1998-12-23T05:00:00+0000,nyt://article/aae2ecc4-f440-5989-bdab-6e56f562...,World,nyt://article/aae2ecc4-f440-5989-bdab-6e56f562...,News,The New York Times,A,An ambitious United States effort to help trai...,1261,Foreign Desk,https://www.nytimes.com/1998/12/23/world/us-he...,article,1,U.S. Help for Mexican Military Has Not Curtail...,By Tim Golden


In [7]:
pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())

tokenizer = treebank.TreebankWordTokenizer()

def sentiment(sentence):
  senti=0
  words = [word.lower() for word in tokenizer.tokenize(sentence)]
  for word in words:
    if word in pos_list:
      senti += 1
    elif word in neg_list:
      senti -= 1
  return senti

In [8]:
nyt['sentiment']=nyt['abstract'].apply(sentiment)

In [9]:
nyt.head()

,snippet,abstract,pub_date,_id,section_name,uri,type_of_material,source,print_section,lead_paragraph,word_count,news_desk,web_url,document_type,print_page,headline,author,sentiment
0,Mexican lawmakers reach compromise on country'...,Mexican lawmakers reach compromise on country'...,1998-12-31T05:00:00+0000,nyt://article/6238be98-a87a-5fbd-8a5d-20b15341...,World,nyt://article/6238be98-a87a-5fbd-8a5d-20b15341...,News,The New York Times,A,After weeks of tense impasse and with less tha...,518,Foreign Desk,https://www.nytimes.com/1998/12/31/world/just-...,article,9,"Just Hours Before Deadline, Mexicans Draft an ...",By Ginger Thompson,0
1,Three new studies report new technique to pinp...,Three new studies report new technique to pinp...,1998-12-29T05:00:00+0000,nyt://article/bd31e0fe-4053-55f8-9e24-86b10281...,Science,nyt://article/bd31e0fe-4053-55f8-9e24-86b10281...,News,The New York Times,F,"Every winter, after flying south from the East...",1170,Science Desk,https://www.nytimes.com/1998/12/29/science/on-...,article,5,"On the Trail of the Monarch, With the Aid of C...",By Carol Kaesuk Yoon,2
2,Interview with Lieut Col Hildegardo Bacillo Go...,Interview with Lieut Col Hildegardo Bacillo Go...,1998-12-29T05:00:00+0000,nyt://article/fc8cfb88-aa41-5ec5-8328-f2e6d870...,World,nyt://article/fc8cfb88-aa41-5ec5-8328-f2e6d870...,News,The New York Times,A,Sheltered from view by bushes on the side of t...,1352,Foreign Desk,https://www.nytimes.com/1998/12/29/world/mexic...,article,3,Mexican Army Protester Goes Loudly Into Hiding,By Ginger Thompson,-3
3,"Some 2,000 people gather at graves of 45 peopl...","Some 2,000 people gather at graves of 45 peopl...",1998-12-23T05:00:00+0000,nyt://article/1eaf78c0-88df-5390-bd95-664fcf07...,World,nyt://article/1eaf78c0-88df-5390-bd95-664fcf07...,News,The New York Times,A,"For six days, only the dogs and chickens were ...",951,Foreign Desk,https://www.nytimes.com/1998/12/23/world/actea...,article,4,Acteal Journal; Where Killings Defiled a Churc...,By Ginger Thompson,-5
4,Ambitious United States effort to help train a...,Ambitious United States effort to help train a...,1998-12-23T05:00:00+0000,nyt://article/aae2ecc4-f440-5989-bdab-6e56f562...,World,nyt://article/aae2ecc4-f440-5989-bdab-6e56f562...,News,The New York Times,A,An ambitious United States effort to help trai...,1261,Foreign Desk,https://www.nytimes.com/1998/12/23/world/us-he...,article,1,U.S. Help for Mexican Military Has Not Curtail...,By Tim Golden,2
